# 4colab

In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# result_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/result'

In [2]:
data_dir = 'data'
result_dir = 'result'

# Import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [4]:
af = 'vv'
pf = '__1__'

Xtrain = np.load(f'{data_dir}/{af}3.tr.X.{pf}.npy', allow_pickle=True)
y_train = np.load(f'{data_dir}/{af}3.tr.y.npy', allow_pickle=True)

Xtest = np.load(f'{data_dir}/{af}3.t.X.{pf}.npy', allow_pickle=True)
y_test = np.load(f'{data_dir}/{af}3.t.y.npy', allow_pickle=True)

# Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
# y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [5]:
Xtrain.shape

(1529165, 63)

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [7]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    #? relabel to use classification metrics
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [8]:
import pickle

def save_result(model, Xtr, Xt, yt_pred, scores, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save prediction
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.result.t.y_pred.npy', yt_pred)
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.result.t.scores.npy', scores)

    #? save transformed X as well
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.tr.X.npy', Xtr)
    np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.t.X.npy', Xt)
    # np.save(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.data.v.X.npy', Xv)

    #? save fts
    open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}5.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

## Exp 01: Run on original features (0-8)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`

Using only numeric features from original data, the result is not good

```
CPU times: total: 45min 21s
Wall time: 3min 16s

CPU times: total: 24min 33s
Wall time: 1min 39s

[[752922   8658]
 [  2860    102]]
              precision    recall  f1-score   support

           0     0.9962    0.9886    0.9924    761580
           1     0.0116    0.0344    0.0174      2962

    accuracy                         0.9849    764542
   macro avg     0.5039    0.5115    0.5049    764542
weighted avg     0.9924    0.9849    0.9886    764542

0.5115338604758198
```

In [9]:
""" select features """
selected_fts_names__01 = fts_names[0:8]
', '.join(selected_fts_names__01)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec'

In [10]:
X_train__01 = Xtrain[:,0:8]
X_test__01 = Xtest[:,0:8]
# X_val__01 = Xval[:,0:8]

print(X_train__01.shape)

(1529165, 8)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__01 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=-1)

%time model__01.fit(X_train__01)

CPU times: total: 45min 21s
Wall time: 3min 16s


LocalOutlierFactor(contamination=0.01, n_jobs=-1, n_neighbors=35, novelty=True)

In [12]:
%%time
""" predict """
y_pred__01 = model__01.predict(X_test__01)

CPU times: total: 24min 33s
Wall time: 1min 39s


In [13]:
""" evaluate """
roc_auc__01 = evaluate(y_test, y_pred__01)

[[752922   8658]
 [  2860    102]]
              precision    recall  f1-score   support

           0     0.9962    0.9886    0.9924    761580
           1     0.0116    0.0344    0.0174      2962

    accuracy                         0.9849    764542
   macro avg     0.5039    0.5115    0.5049    764542
weighted avg     0.9924    0.9849    0.9886    764542

0.5115338604758198


In [14]:
""" get lof scores """
scores_pred__01 = model__01.score_samples(X_test__01)

In [15]:
""" save model """
save_result(model__01, X_train__01, X_test__01, y_pred__01, scores_pred__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [16]:
""" clean """
del X_train__01, X_test__01#, X_val__01
del model__01, y_pred__01, scores_pred__01#, yval_pred__01

## Exp 02: Run on numeric features (0-14)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `TotPkts`, `TotBytes`, `SrcBytes`, `DstBytes`, `DstBytesPerSec`

The result slightly increases compared to Exp01.  

```
CPU times: total: 56min 37s
Wall time: 4min 16s

CPU times: total: 30min 26s
Wall time: 2min 12s

[[749817  11763]
 [  2662    300]]
              precision    recall  f1-score   support

           0     0.9965    0.9846    0.9905    761580
           1     0.0249    0.1013    0.0399      2962

    accuracy                         0.9811    764542
   macro avg     0.5107    0.5429    0.5152    764542
weighted avg     0.9927    0.9811    0.9868    764542

0.5429186978973082
```

In [9]:
""" select features """
selected_fts_names__02 = fts_names[0:14]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, TotPkts, TotBytes, SrcBytes, DstBytes, DstBytesPerSec'

In [10]:
X_train__02 = Xtrain[:,0:14]
X_test__02 = Xtest[:,0:14]
# X_val__02 = Xval[:,0:14]

print(X_train__02.shape)

(1529165, 14)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__02 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=-1)

%time model__02.fit(X_train__02)

CPU times: total: 56min 37s
Wall time: 4min 16s


LocalOutlierFactor(contamination=0.01, n_jobs=-1, n_neighbors=35, novelty=True)

In [12]:
%%time
""" predict """
y_pred__02 = model__02.predict(X_test__02)

CPU times: total: 30min 26s
Wall time: 2min 12s


In [13]:
""" evaluate """
roc_auc__02 = evaluate(y_test, y_pred__02)

[[749817  11763]
 [  2662    300]]
              precision    recall  f1-score   support

           0     0.9965    0.9846    0.9905    761580
           1     0.0249    0.1013    0.0399      2962

    accuracy                         0.9811    764542
   macro avg     0.5107    0.5429    0.5152    764542
weighted avg     0.9927    0.9811    0.9868    764542

0.5429186978973082


In [14]:
""" get lof scores """
scores_pred__02 = model__02.score_samples(X_test__02)

In [15]:
""" save model """
save_result(model__02, X_train__02, X_test__02, y_pred__02, scores_pred__02, 'lof', selected_fts_names__02, roc_auc__02, 'exp02_fts-0-14', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [16]:
""" clean """
del X_train__02, X_test__02#, X_val__02
del model__02, y_pred__02, scores_pred__02#, yval_pred__02

## Exp 03: Run on all features

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `TotPkts`, `TotBytes`, `SrcBytes`, `DstBytes`, `DstBytesPerSec`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result increase, yet running time is longer  

```
CPU times: total: 7h 49min 9s
Wall time: 34min 52s

CPU times: total: 3h 56min 39s
Wall time: 17min 39s

[[747045  14535]
 [  1239   1723]]
              precision    recall  f1-score   support

           0     0.9983    0.9809    0.9896    761580
           1     0.1060    0.5817    0.1793      2962

    accuracy                         0.9794    764542
   macro avg     0.5522    0.7813    0.5844    764542
weighted avg     0.9949    0.9794    0.9864    764542

0.7813081151929802
```

In [9]:
"', '".join(fts_names)

"Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'TotPkts', 'TotBytes', 'SrcBytes', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other', 'sTos_0', 'sTos_2', 'sTos_3', 'sTos_1', 'sTos_-1', 'dTos_0', 'dTos_-1', 'dTos_3', 'dTos_2', 'dTos_1"

In [10]:
""" select features """
selected_fts_names__03 = ['Dur', 'sTos', 'dTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'DstBytes', 'DstBytesPerSec', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__03 = [fts_names.index(s) for s in selected_fts_names__03]
len(idx__selected_fts_names__03)

53

In [11]:
X_train__03 = Xtrain[:,idx__selected_fts_names__03]
X_test__03 = Xtest[:,idx__selected_fts_names__03]
# X_val__03 = Xval[:,idx__selected_fts_names__03]

print(X_train__03.shape)

(1529165, 53)


In [12]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__03 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=14)

%time model__03.fit(X_train__03)

CPU times: total: 7h 49min 9s
Wall time: 34min 52s


LocalOutlierFactor(contamination=0.01, n_jobs=14, n_neighbors=35, novelty=True)

In [13]:
%%time
""" predict """
y_pred__03 = model__03.predict(X_test__03)

CPU times: total: 3h 56min 39s
Wall time: 17min 39s


In [14]:
""" evaluate """
roc_auc__03 = evaluate(y_test, y_pred__03)

[[747045  14535]
 [  1239   1723]]
              precision    recall  f1-score   support

           0     0.9983    0.9809    0.9896    761580
           1     0.1060    0.5817    0.1793      2962

    accuracy                         0.9794    764542
   macro avg     0.5522    0.7813    0.5844    764542
weighted avg     0.9949    0.9794    0.9864    764542

0.7813081151929802


In [15]:
""" get lof scores """
scores_pred__03 = y_pred__03#model__03.score_samples(X_test__03)

In [16]:
""" save model """
save_result(model__03, X_train__03, X_test__03, y_pred__03, scores_pred__03, 'lof', selected_fts_names__03, roc_auc__03, 'exp03_fts-all', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [17]:
""" clean """
del X_train__03, X_test__03#, X_val__03
del model__03, y_pred__03, scores_pred__03#, yval_pred__03

## Exp 04: Run on chosen set of features

`sTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The overall ROC_AUC end execution time does not change much from experiment 02 or 03.  
Recall for minority class increases.  

```
CPU times: total: 6h 17min 14s
Wall time: 24min 36s

CPU times: total: 3h 20min 18s
Wall time: 13min 30s

[[746158  15422]
 [  1703   1259]]
              precision    recall  f1-score   support

           0     0.9977    0.9797    0.9887    761580
           1     0.0755    0.4251    0.1282      2962

    accuracy                         0.9776    764542
   macro avg     0.5366    0.7024    0.5584    764542
weighted avg     0.9941    0.9776    0.9853    764542

0.7024003174465878
```

In [17]:
""" select features """
selected_fts_names__04 = ['sTos', 'Sport', 'Dport', 'TotPkts', 'TotBytes', 'SrcBytes', 'PktsPerSec', 'BytesPerSec', 'SrcBytesPerSec', 'BytesPerPkt', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__04 = [fts_names.index(s) for s in selected_fts_names__04]
len(idx__selected_fts_names__04)

49

In [18]:
X_train__04 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
X_test__04 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)
# X_val__04 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__04]), axis=1)

print(X_train__04.shape)

(1529165, 49)


In [19]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__04 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=10)

%time model__04.fit(X_train__04)

CPU times: total: 6h 17min 14s
Wall time: 24min 36s


LocalOutlierFactor(contamination=0.01, n_jobs=10, n_neighbors=35, novelty=True)

In [20]:
%%time
""" predict """
y_pred__04 = model__04.predict(X_test__04)

CPU times: total: 3h 20min 18s
Wall time: 13min 30s


In [21]:
""" evaluate """
roc_auc__04 = evaluate(y_test, y_pred__04)

[[746158  15422]
 [  1703   1259]]
              precision    recall  f1-score   support

           0     0.9977    0.9797    0.9887    761580
           1     0.0755    0.4251    0.1282      2962

    accuracy                         0.9776    764542
   macro avg     0.5366    0.7024    0.5584    764542
weighted avg     0.9941    0.9776    0.9853    764542

0.7024003174465878


In [22]:
""" get lof scores """
scores_pred__04 = model__04.score_samples(X_test__04)

In [23]:
""" save model """
save_result(model__04, X_train__04, X_test__04, y_pred__04, scores_pred__04, 'lof', selected_fts_names__04, roc_auc__04, 'exp04_play', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [24]:
""" clean """
del X_train__04, X_test__04#, X_val__04
del model__04, y_pred__04, scores_pred__04#, yval_pred__04

## Exp 05: Run on numeric features chosen from MI + one-hot features

**Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `TotBytes`, `DstBytes`**  

```
CPU times: total: 6h 24min 42s
Wall time: 45min 26s

CPU times: total: 3h 27min 27s
Wall time: 14min

[[747340  14240]
 [  1224   1738]]
              precision    recall  f1-score   support

           0     0.9984    0.9813    0.9898    761580
           1     0.1088    0.5868    0.1835      2962

    accuracy                         0.9798    764542
   macro avg     0.5536    0.7840    0.5866    764542
weighted avg     0.9949    0.9798    0.9866    764542

0.7840338644212053
```

In [9]:
""" select features """
selected_fts_names__05 = ['dTos', 'Sport', 'BytesPerPkt', 'Dur', 'BytesPerSec', 'DstBytesPerSec', 'PktsPerSec', 'SrcBytesPerSec', 'TotBytes', 'DstBytes', 'State_CON', 'State_alltcp', 'State_INT', 'State_S_', 'State_URP', 'State_ECO', 'State_RED', 'State_REQ', 'State_ECR', 'State_URH', 'State_TXD', 'State_URFIL', 'State_R_', 'State_URN', 'State_RSP', 'State_URHPRO', 'State_A_', 'State_other', 'Flag_nan', 'Flag_S', 'Flag_A', 'Flag_P', 'Flag_R', 'Flag_F', 'Proto_udp', 'Proto_tcp', 'Proto_icmp', 'Proto_rtp', 'Proto_rtcp', 'Proto_igmp', 'Proto_arp', 'Proto_other', 'Service_80', 'Service_443', 'Service_21', 'Service_22', 'Service_25', 'Service_6667', 'Service_other']
idx__selected_fts_names__05 = [fts_names.index(s) for s in selected_fts_names__05]
len(idx__selected_fts_names__05)

49

In [10]:
X_train__05 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
X_test__05 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)
# X_val__05 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__05]), axis=1)

print(X_train__05.shape)

(1529165, 49)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__05 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=-1)

%time model__05.fit(X_train__05)

CPU times: total: 6h 24min 42s
Wall time: 45min 26s


LocalOutlierFactor(contamination=0.01, n_jobs=-1, n_neighbors=35, novelty=True)

In [12]:
%%time
""" predict """
y_pred__05 = model__05.predict(X_test__05)

CPU times: total: 3h 27min 27s
Wall time: 14min


In [13]:
""" evaluate """
roc_auc__05 = evaluate(y_test, y_pred__05)

[[747340  14240]
 [  1224   1738]]
              precision    recall  f1-score   support

           0     0.9984    0.9813    0.9898    761580
           1     0.1088    0.5868    0.1835      2962

    accuracy                         0.9798    764542
   macro avg     0.5536    0.7840    0.5866    764542
weighted avg     0.9949    0.9798    0.9866    764542

0.7840338644212053


In [14]:
""" get lof scores """
scores_pred__05 = model__05.score_samples(X_test__05)

In [15]:
""" save model """
save_result(model__05, X_train__05, X_test__05, y_pred__05, scores_pred__05, 'lof', selected_fts_names__05, roc_auc__05, 'exp05_mi1', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [16]:
""" clean """
del X_train__05, X_test__05#, X_val__05
del model__05, y_pred__05, scores_pred__05#, yval_pred__05

## Exp 06: Run on features chosen from MI

`State_S_` ,` dTos` ,` Proto_tcp` ,` Proto_udp` ,` Service_80` ,` Service_other` ,` State_CON` ,` Service_25` ,` Sport` ,` Flag_S` ,` Flag_R` ,` BytesPerPkt` ,` Dur` ,` Service_6667` ,` BytesPerSec` ,` State_alltcp` ,` DstBytesPerSec` ,` Flag_A` ,` PktsPerSec` ,` SrcBytesPerSec`

```
CPU times: total: 4h 50min 59s
Wall time: 21min 39s

CPU times: total: 2h 56min 39s
Wall time: 13min 42s

[[748829  12751]
 [  1489   1473]]
              precision    recall  f1-score   support

           0     0.9980    0.9833    0.9906    761580
           1     0.1036    0.4973    0.1714      2962

    accuracy                         0.9814    764542
   macro avg     0.5508    0.7403    0.5810    764542
weighted avg     0.9946    0.9814    0.9874    764542

0.7402781490429675
```

In [9]:
""" select features """
# selected_fts_names__06 = fts_names[29:30] + fts_names[2:3] + fts_names[51:52] + fts_names[50:51] + fts_names[58:59] + fts_names[64:65] + fts_names[26:27] + fts_names[62:63] + fts_names[3:4] + fts_names[45:46] + fts_names[48:49] + fts_names[26:65]
selected_fts_names__06 = ['State_S_','dTos','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Sport','Flag_S','Flag_R','BytesPerPkt','Dur','Service_6667','BytesPerSec','State_alltcp','DstBytesPerSec','Flag_A','PktsPerSec','SrcBytesPerSec']
idx__selected_fts_names__06 = [fts_names.index(s) for s in selected_fts_names__06]
len(idx__selected_fts_names__06)
# ', '.join(selected_fts_names__05)

20

In [10]:
X_train__06 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
X_test__06 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)
# X_val__06 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__06]), axis=1)

print(X_train__06.shape)

(1529165, 20)


In [11]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__06 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=10)

%time model__06.fit(X_train__06)

CPU times: total: 4h 50min 59s
Wall time: 21min 39s


LocalOutlierFactor(contamination=0.01, n_jobs=10, n_neighbors=35, novelty=True)

In [12]:
%%time
""" predict """
y_pred__06 = model__06.predict(X_test__06)

CPU times: total: 2h 56min 39s
Wall time: 13min 42s


In [13]:
""" evaluate """
roc_auc__06 = evaluate(y_test, y_pred__06)

[[748829  12751]
 [  1489   1473]]
              precision    recall  f1-score   support

           0     0.9980    0.9833    0.9906    761580
           1     0.1036    0.4973    0.1714      2962

    accuracy                         0.9814    764542
   macro avg     0.5508    0.7403    0.5810    764542
weighted avg     0.9946    0.9814    0.9874    764542

0.7402781490429675


In [14]:
""" get lof scores """
scores_pred__06 = model__06.score_samples(X_test__06)

In [15]:
save_result(model__06, X_train__06, X_test__06, y_pred__06, scores_pred__06, 'lof', selected_fts_names__06, roc_auc__06, 'exp06_mi2', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [16]:
""" clean """
del X_train__06, X_test__06#, X_val__06
del model__06, y_pred__06, scores_pred__06#, yval_pred__06

## Exp 07: Run on numeric features chosen from MI + ohe features

Chosen features from MI: `dTos`, `Sport`, `BytesPerPkt`, `Dur`, `BytesPerSec`, `DstBytesPerSec`, `PktsPerSec`, `SrcBytesPerSec`, `State_S_`, `Proto_tcp`, `Proto_udp`, `Service_80`, `Service_other`, `State_CON`, `Service_25`, `Flag_S`

```
CPU times: total: 5h 51min 49s
Wall time: 28min 23s

CPU times: total: 2h 40min 3s
Wall time: 12min 20s

[[749790  11790]
 [  1489   1473]]
              precision    recall  f1-score   support

           0     0.9980    0.9845    0.9912    761580
           1     0.1111    0.4973    0.1816      2962

    accuracy                         0.9826    764542
   macro avg     0.5545    0.7409    0.5864    764542
weighted avg     0.9946    0.9826    0.9881    764542

0.7409090742248261
```

In [17]:
""" select features """
selected_fts_names__07 = ['dTos','Sport','BytesPerPkt','Dur','BytesPerSec','DstBytesPerSec','PktsPerSec','SrcBytesPerSec',
    'State_S_','Proto_tcp','Proto_udp','Service_80','Service_other','State_CON','Service_25','Flag_S'
]
idx__selected_fts_names__07 = [fts_names.index(s) for s in selected_fts_names__07]
len(idx__selected_fts_names__07)

16

In [18]:
X_train__07 = np.concatenate(tuple([Xtrain[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
X_test__07 = np.concatenate(tuple([Xtest[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)
# X_val__07 = np.concatenate(tuple([Xval[:,[idx]] for idx in idx__selected_fts_names__07]), axis=1)

print(X_train__07.shape)

(1529165, 16)


In [19]:
""" train """
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 35
contamination = 0.01
model__07 = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, novelty=True, n_jobs=10)

%time model__07.fit(X_train__07)

CPU times: total: 5h 51min 49s
Wall time: 28min 23s


LocalOutlierFactor(contamination=0.01, n_jobs=10, n_neighbors=35, novelty=True)

In [20]:
%%time
""" predict """
y_pred__07 = model__07.predict(X_test__07)

CPU times: total: 2h 40min 3s
Wall time: 12min 20s


In [21]:
""" evaluate """
roc_auc__07 = evaluate(y_test, y_pred__07)

[[749790  11790]
 [  1489   1473]]
              precision    recall  f1-score   support

           0     0.9980    0.9845    0.9912    761580
           1     0.1111    0.4973    0.1816      2962

    accuracy                         0.9826    764542
   macro avg     0.5545    0.7409    0.5864    764542
weighted avg     0.9946    0.9826    0.9881    764542

0.7409090742248261


In [22]:
""" get lof scores """
scores_pred__07 = model__07.score_samples(X_test__07)

In [23]:
""" save model """
save_result(model__07, X_train__07, X_test__07, y_pred__07, scores_pred__07, 'lof', selected_fts_names__07, roc_auc__07, 'exp07_mi3', config=f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [24]:
""" clean """
del X_train__07, X_test__07#, X_val__07
del model__07, y_pred__07, scores_pred__07#, yval_pred__07

## Checking score

The best set of features is of Exp06.  
Use this experiment result to further analyze the scores of outliers

Check `vv2-6.after_predict`